In [1]:
import pandas as pd
import numpy as np
import imaplib
import email
import quopri
import re
import html
from bs4 import BeautifulSoup
import requests

In [2]:
username = 'prospectstudent@colby.edu'
password = 'Student.2017'

# username = 'haohaom2@gmail.com'
# password = '62947394'

### Accessing the email

In [3]:
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(username, password)
mail.list()
# Out: list of "folders" aka labels in gmail.
mail.select("inbox") # connect to inbox.

('OK', [b'4007'])

In [27]:
result, data = mail.uid('search', None, "ALL") # search and return uids instead

first_email_id = data[0].split()[0]
latest_email_uid = data[0].split()[-1]

test_email_uid = data[0].split()[-10]

ids = data[0].split()

In [5]:
print('first email uid ', first_email_id)
print('latest_email_uid ', latest_email_uid)

first email uid  b'4009'
latest_email_uid  b'12244'


### Reading the email message

In [28]:
result, data = mail.uid('fetch', test_email_uid, '(RFC822)')
raw_email = str(data[0][1], 'utf-8')
email_message = email.message_from_string(raw_email)
print(email_message['subject'])

Google Alert - "Patricia Smith"


In [29]:
def get_text_from_email(email_msg):
    if email_msg.is_multipart():
        s = ''
        for payload in email_msg.get_payload():
            s += payload.get_payload()
    else:
        s = b.get_payload()
    
    return s

In [34]:
# This cleans html stuff so I can extract the xml code
text = get_text_from_email(email_message).replace('\r\n', '').replace('=', '')

In [36]:
# text = html.unescape(text)
text

'PT09IE5ld3MgLSA2IG5ldyByZXN1bHRzIGZvciBbIlBhdHJpY2lhIFNtaXRoIl0gPT09DQoNCkJyb29rbHluIHdvbWFuIGdhdmUgbm8gaGludHMgb2YgdHJvdWJsZSBvciBlbmVtaWVzIGJlZm9yZSBmb3VuZCBkaXNtZW1iZXJlZA0KaW4gcGFyazsgZmFtaWx5IC4uLg0KTmV3IFlvcmsgRGFpbHkgTmV3cw0KTG9jYWwgZG9nIHdhbGtlciBQYXRyaWNpYSBTbWl0aCBzcG90dGVkIHRoZSB3b21hbidzIGJvZHkg4oCUIGEgdG9yc28gd2l0aG91dA0KYXJtcyBhbmQgbGVncyDigJQgaW4gYSBwaWxlIG9mIGxlYXZlcyBieSBhIG5hcnJvdyBwYXRod2F5IHRoYXQgd2luZHMgdGhyb3VnaA0KdGhlIHBhcmsuIFNtaXRoIHdlbnQgZG93biB0aGUgc2FtZSByb2FkIHdpdGggaGVyIFBla2luZ2VzZSBwdXAgdGhlIGV2ZW5pbmcNCmJlZm9yZSDigJQgYnV0IHRoaXMgdGltZSB0aGVyZSB3YXMgc29tZXRoaW5nIGRpZmZlcmVudC4g4oCcSSBzdG9wcGVkIGFuZCAuLi4NCjxodHRwczovL3d3dy5nb29nbGUuY29tL3VybD9yY3Q9aiZzYT10JnVybD1odHRwOi8vd3d3Lm55ZGFpbHluZXdzLmNvbS9uZXcteW9yay93b21hbi1nYXZlLW5vLWhpbnRzLXRyb3VibGUtZm91bmQtYnV0Y2hlcmVkLXBhcmstYXJ0aWNsZS0xLjM5NTUwMzUmY3Q9Z2EmY2Q9Q0FFWUFDb1VNVE0wTWprMk1qQTFOekUwT0RBek9ETTJNamt5R2pNMFkyUTRZelUzWWpSbVl6UTNNVEk2WTI5dE9tVnVPbFZUJnVzZz1BRlFqQ05Ia05ZNTN6WW1FeVhodUR2Qjc5ZmNwVUJzdy1

In [61]:
# extracts xml files from the raw string text
def extract_xml(raw_text):
    raw_text = raw_text.replace('\r\n', '').replace('=', '')
    m = re.findall('alerts/feeds\S+', raw_text)
    
    links = [('https://www.google.com/' + x).replace('\"', '') for x in m]
    return links

### Using Regex to get the links

In [126]:
# Passes in an xml link
# returns a list of links used in the website
def get_url_from_xml(xml_link):
    r  = requests.get(xml_link)
    soup = BeautifulSoup(r.content, 'html.parser')   # creates a BS4 from given xml link
    web_lists = soup.find_all('link')[1:]            # finds all the BS4 links
    
    urls = [re.findall('url=(.*?)&', url['href'])[0] for url in web_lists]
    return urls

In [127]:
get_url_from_xml(extract_xml(text)[0])

['http://www.nydailynews.com/new-york/woman-gave-no-hints-trouble-found-butchered-park-article-1.3955035',
 'http://www.myarklamiss.com/news/local-news/la-state-bill-passes-to-require-recess-for-students-k-8/1142778829',
 'https://www.theadvertiser.com/story/news/local/louisiana/2018/04/25/bring-back-recess-bill-others-clear-house-committee/552368002/',
 'https://www.ocregister.com/2018/04/25/books-authors-and-readers-free-events-happening-at-litfest-pasadena/',
 'http://www.kalb.com/content/news/House-education-committee-approves-bill-to-require-unstructured-playtime-480867921.html',
 'http://www.theadvocate.com/baton_rouge/news/politics/legislature/article_4bf71aea-47fe-11e8-8333-97df9fd80aff.html']

### Scrapes each link and creates a keyward card

In [64]:

m = re.findall('alerts/feeds\S+', text)
url = ('https://www.google.com/' + m[0]).replace('\"', '')
url

'https://www.google.com/alerts/feeds/16218955178302834825/3213032149530912798'

In [82]:
r  = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [84]:
# use &ct as a stopper

web_lists = soup.find_all('link')[1:]
for web in web_lists:
    print(web['href'])

https://www.google.com/url?rct=j&sa=t&url=http://www.nydailynews.com/new-york/woman-gave-no-hints-trouble-found-butchered-park-article-1.3955035&ct=ga&cd=CAIyGjM0Y2Q4YzU3YjRmYzQ3MTI6Y29tOmVuOlVT&usg=AFQjCNHkNY53zYmEyXhuDvB79fcpUBsw-w
https://www.google.com/url?rct=j&sa=t&url=http://www.myarklamiss.com/news/local-news/la-state-bill-passes-to-require-recess-for-students-k-8/1142778829&ct=ga&cd=CAIyGjM0Y2Q4YzU3YjRmYzQ3MTI6Y29tOmVuOlVT&usg=AFQjCNGzPeUhmPb0LZifoZLiXkN5K5t7vQ
https://www.google.com/url?rct=j&sa=t&url=https://www.theadvertiser.com/story/news/local/louisiana/2018/04/25/bring-back-recess-bill-others-clear-house-committee/552368002/&ct=ga&cd=CAIyGjM0Y2Q4YzU3YjRmYzQ3MTI6Y29tOmVuOlVT&usg=AFQjCNHE3EH1EjBAaOCAEMwdtG2IS54gPg
https://www.google.com/url?rct=j&sa=t&url=https://www.ocregister.com/2018/04/25/books-authors-and-readers-free-events-happening-at-litfest-pasadena/&ct=ga&cd=CAIyGjM0Y2Q4YzU3YjRmYzQ3MTI6Y29tOmVuOlVT&usg=AFQjCNHQFFWwUH7giFiK_y-z2WsDq5n8YQ
https://www.google.com/ur

In [105]:
url = web_lists[0]['href']
url

'https://www.google.com/url?rct=j&sa=t&url=http://www.nydailynews.com/new-york/woman-gave-no-hints-trouble-found-butchered-park-article-1.3955035&ct=ga&cd=CAIyGjM0Y2Q4YzU3YjRmYzQ3MTI6Y29tOmVuOlVT&usg=AFQjCNHkNY53zYmEyXhuDvB79fcpUBsw-w'

In [121]:
[re.findall('url=(.*?)&', url['href'])[0] for url in web_lists]

['http://www.nydailynews.com/new-york/woman-gave-no-hints-trouble-found-butchered-park-article-1.3955035',
 'http://www.myarklamiss.com/news/local-news/la-state-bill-passes-to-require-recess-for-students-k-8/1142778829',
 'https://www.theadvertiser.com/story/news/local/louisiana/2018/04/25/bring-back-recess-bill-others-clear-house-committee/552368002/',
 'https://www.ocregister.com/2018/04/25/books-authors-and-readers-free-events-happening-at-litfest-pasadena/',
 'http://www.kalb.com/content/news/House-education-committee-approves-bill-to-require-unstructured-playtime-480867921.html',
 'http://www.theadvocate.com/baton_rouge/news/politics/legislature/article_4bf71aea-47fe-11e8-8333-97df9fd80aff.html']

In [110]:
test_soup = BeautifulSoup(requests.get(url).content, 'html.parser')

In [111]:
test_soup.get_text()

'window.googleJavaScriptRedirect=1var n={navigateTo:function(b,a,d){if(b!=a&&b.google){if(b.google.r){b.google.r=0;b.location.href=d;a.location.replace("about:blank");}}else{a.location.replace(d);}}};n.navigateTo(window.parent,window,"http://www.nydailynews.com/new-york/woman-gave-no-hints-trouble-found-butchered-park-article-1.3955035");\n'

In [89]:
def get_first_text_block(email_message_instance):
    maintype = email_message_instance.get_content_maintype()
    if maintype == 'multipart':
        for part in email_message_instance.get_payload():
            if part.get_content_maintype() == 'text':
                return part.get_payload()
    elif maintype == 'text':
        return email_message_instance.get_payload()

In [107]:
# get_first_text_block(email_message)

#### using mail.search()

In [70]:
type, data = mail.search(None, 'ALL')
mail_ids = data[0].decode()
id_list = mail_ids.split()

In [71]:
first_email_id = int(id_list[0])
latest_email_id = int(id_list[-1])

In [89]:
typ, data = mail.fetch('4', '(RFC822)' )

In [91]:
for response_part in data:
    if isinstance(response_part, tuple):
        msg = email.message_from_string(str(response_part[1]))
        email_subject = msg['subject']
        email_from = msg['from']
        print( 'From : ' + str(email_from) + '\n')
        print ('Subject : ' + str(email_subject) + '\n')

From : None

Subject : None



In [97]:
raw_email = data[0][1]
email_message = email.message_from_string(str(raw_email))
print(email_message['To'])

None


In [84]:
for response_part in data:
    if isinstance(response_part, tuple):
        print(type(response_part[1]))

<class 'bytes'>


In [65]:
import smtplib
import time
import imaplib
import email
def read_email_from_gmail(user, pw):
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(user, pw)
        mail.select('inbox')

        typ, data = mail.search(None, 'ALL')
        mail_ids = data[0]

        id_list = mail_ids.split()   
        first_email_id = int(id_list[0])
        latest_email_id = int(id_list[-1])


        for i in range(latest_email_id,first_email_id, -1):
            typ, data = mail.fetch(str(i), '(RFC822)' )

            for response_part in data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_string(response_part[1])
                    email_subject = msg['subject']
                    email_from = msg['from']
                    print( 'From : ' + email_from + '\n')
                    print ('Subject : ' + email_subject + '\n')

    except Exception as e:
        print (e)

In [66]:
read_email_from_gmail(username, password)

initial_value must be str or None, not bytes
